In [1]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

In [2]:
# Load a specific run
run_id = "baseline-tdms-valid_llama3_70b-05212024-075640"
path = f"results/{run_id}/df.feather"

import pandas as pd
df = pd.read_feather(path)


In [3]:
# Load ground truth
from src.dataset import TDMSDataset, PATH, UNANSWERABLE, LogResult
valid_dataset = TDMSDataset(PATH.VAL)
df_ground_truth = pd.DataFrame([{"f":f, "ground_truth":gt} for f, _, gt in [valid_dataset.__getitem__(i) for i in range(len(df))]])
df = df_ground_truth.merge(df, on="f")

In [4]:
len(df)

100

In [5]:
# evaluate
from scoring_program.evaluation import Metrics

results = Metrics.evaluate_property_wise_json_based(
    label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"])
)

results.update(
    Metrics.evaluate_rouge(label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"]))
)
pd.DataFrame([results]).T

2024-05-21 13:15:15.727165: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/user/.venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,0
general_accuracy,80.00
exact_recalls_task,30.05
exact_recalls_dataset,40.39
exact_recalls_metric,29.06
exact_recalls_Score,9.85
exact_recalls_overall,27.34
partial_recalls_task,44.83
partial_recalls_dataset,48.77
partial_recalls_metric,36.45
partial_recalls_Score,12.14


In [ ]:
# confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(df["ground_truth"], df["pred"])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()